# 0 导入必要的包

In [2]:
import numpy as np
import pandas as pd
import random
import sys
import os
from tqdm import tqdm
sys.path.append("../../tools/")
import commontools
import ucTools
import funclib
sys.path.append("../../")
import train
import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. 读取数据

In [71]:
# 1515 数据
sprot1515 = pd.read_csv('./data/ecoli1515/1515testset_with_unirep.tsv',  sep='\t', index_col=0)

In [91]:
colname =  [['id', 'ec_number', 'seq', 'isemzyme', 'isMultiFunctional', 'blattner_id', 'blattner_ec_number' ] +['f'+str(i) for i in range(1,1901)]]
sprot1515.columns =colname

In [92]:
sprot1515

,id,ec_number,seq,isemzyme,isMultiFunctional,blattner_id,blattner_ec_number,f1,f2,f3,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,P42641,3.6.5.-,MKFVDEASILVVAGDGGNGCVSFRREKYIPKGGPDGGDGGDGGDVW...,True,False,b3183,NaN,0.003056,-0.087184,0.043387,...,0.000332,-0.065542,0.117107,-0.236777,-0.622398,0.012113,0.010506,0.058984,0.061533,0.968014
1,P0AE22,3.1.3.2,MRKITQAISAVCLLFALNSSAVALASSPSPLNPGTNVARLAEQAPI...,True,False,b4055,3.1.3.2,0.001026,0.618393,0.409527,...,0.030307,-0.244636,-0.038585,0.151263,-0.020389,0.095321,-0.031043,-0.881730,-0.196211,0.054455
2,P21499,3.1.13.1,MSQDPFQEREAEKYANPIPSREFILEHLTKREKPASRDELAVELHI...,True,False,b4179,3.1.-.-,0.073632,-0.003258,0.008865,...,0.002123,0.060351,-0.008826,0.006141,-0.129352,-0.075010,0.023712,-0.002246,0.893310,0.140025
3,P39286,3.6.1.-,MSKNKLSKGQQRRVNANHQRRLKTSKEKPDYDDNLFGEPDEGIVIS...,True,False,b4161,3.6.1.-,0.000775,-0.008305,0.041130,...,0.066885,0.020991,0.073577,-0.034962,-0.036755,0.053803,-0.017355,0.100456,-0.202344,0.351100
4,P56100,7.1.1.7,MWYFAWILGTLLACSFGVITALALEHVESGKAGQEDI,True,False,b4515,1.10.3.10,0.001811,-0.050181,0.037575,...,0.042016,-0.028950,0.305668,-0.105587,-0.113769,0.680413,0.138797,-0.268528,-0.076093,-0.038212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,P36837,NaN,MNTTTPMGMLQQPRPFFMIFFVELWERFGYYGVQGVLAVFFVKQLG...,False,False,b3496,NaN,0.000070,-0.021424,0.203301,...,0.005911,-0.016855,0.377301,-0.500727,-0.247861,0.752678,-0.005501,0.396191,-0.215832,-0.062812
4746,P77304,NaN,MSTANQKPTESVSLNAFKQPKAFYLIFSIELWERFGYYGLQGIMAV...,False,False,b1634,NaN,0.008628,-0.065382,0.048911,...,-0.001197,-0.101824,0.013007,-0.304450,-0.295525,-0.124492,0.010846,0.032853,-0.155702,-0.183535
4747,P27248,2.1.2.10,MAQQTPLYEQHTLCGARMVDFHGWMMPLHYGSQIDEHHAVRTDAGM...,True,False,b2905,2.1.2.10,0.006723,0.227349,0.029725,...,-0.124573,-0.032099,-0.170996,0.104425,-0.163041,0.183720,-0.038814,0.036993,-0.044558,0.010447
4748,P0AC88,4.2.1.47,MSKVALITGVTGQDGSYLAEFLLEKGYEVHGIKRRASSFNTERVDH...,True,False,b2053,4.2.1.47,0.000665,-0.029179,0.024183,...,0.145032,0.867074,-0.263008,0.009123,0.036783,-0.073805,0.439632,0.014501,-0.205913,-0.005663


In [66]:
sprot1515.iloc[:,8:1909]

,0,1,2,3,4,5,6,7,8,9,...,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899
0,0.003056,-0.087184,0.043387,-0.018850,-0.623890,0.034710,0.133006,-0.056650,-0.004179,-0.002791,...,0.000332,-0.065542,0.117107,-0.236777,-0.622398,0.012113,0.010506,0.058984,0.061533,0.968014
1,0.001026,0.618393,0.409527,-0.002331,-0.670928,0.036821,-0.148798,-0.038413,-0.002616,0.026154,...,0.030307,-0.244636,-0.038585,0.151263,-0.020389,0.095321,-0.031043,-0.881730,-0.196211,0.054455
2,0.073632,-0.003258,0.008865,-0.012720,-0.072697,-0.026430,-0.033917,-0.000300,-0.001022,0.041117,...,0.002123,0.060351,-0.008826,0.006141,-0.129352,-0.075010,0.023712,-0.002246,0.893310,0.140025
3,0.000775,-0.008305,0.041130,-0.001737,-0.156966,0.009515,0.400612,-0.021405,-0.002408,0.028499,...,0.066885,0.020991,0.073577,-0.034962,-0.036755,0.053803,-0.017355,0.100456,-0.202344,0.351100
4,0.001811,-0.050181,0.037575,-0.005655,0.025414,0.003502,-0.096007,-0.006303,-0.000473,0.012226,...,0.042016,-0.028950,0.305668,-0.105587,-0.113769,0.680413,0.138797,-0.268528,-0.076093,-0.038212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,0.000070,-0.021424,0.203301,0.025054,0.425925,-0.013487,-0.131236,-0.000778,-0.000883,-0.099993,...,0.005911,-0.016855,0.377301,-0.500727,-0.247861,0.752678,-0.005501,0.396191,-0.215832,-0.062812
4746,0.008628,-0.065382,0.048911,-0.215166,-0.197027,0.001949,0.045984,-0.001380,-0.004231,0.022663,...,-0.001197,-0.101824,0.013007,-0.304450,-0.295525,-0.124492,0.010846,0.032853,-0.155702,-0.183535
4747,0.006723,0.227349,0.029725,0.032317,0.064065,0.092823,-0.044260,-0.024524,-0.001700,0.055556,...,-0.124573,-0.032099,-0.170996,0.104425,-0.163041,0.183720,-0.038814,0.036993,-0.044558,0.010447
4748,0.000665,-0.029179,0.024183,-0.005963,-0.175254,0.042627,0.554610,-0.020871,-0.008380,0.105102,...,0.145032,0.867074,-0.263008,0.009123,0.036783,-0.073805,0.439632,0.014501,-0.205913,-0.005663


In [4]:
funclib.table2fasta(sprot1515, './data/sprot1515.fasta')

Write finished


## 2. 序列比对

In [5]:
train = pd.read_hdf('./data/train.h5',key='data')
test = pd.read_hdf('./data/test.h5',key='data')
head = funclib.table_head + ['f'+str(i) for i in range(1, 1901) ]
head = head + ['ec_label','ec_appears']
train.columns = head
test.columns = head

In [ ]:
train = pd.read_hdf('./data/train.h5',key='data')

In [4]:
file_enzyme_train_x = '../../data/preprocess/enzyme_train_x.feather'
file_enzyme_train_y = '../../data/preprocess/enzyme_train_y.feather'
DATE_TRAIN ='2018-01-01'
uctools =  ucTools.ucTools('172.16.25.20')
cnx_ecnumber = uctools.db_conn()

eclablefile='../../data/dict_ec_label.npy'
featurefile='../../data/sprot_unirep.feather'

dict_ec_label, dict_label_ec = train.load_ec_lable_dict(eclablefile)

In [6]:
len(dict_ec_label)

5389

In [59]:
enzyme_X, enzyme_Y = train.get_enzyme_train_set( train_date=DATE_TRAIN, 
                            eclablefile='../../data/dict_ec_label.npy', 
                            featurefile='../../data/sprot_unirep.feather', 
                            cnx=cnx_ecnumber,
                            trainX=file_enzyme_train_x,
                            trainY=file_enzyme_train_y
                          )

In [60]:
file_isenzyme_model = './model/isenzyme.model'
train.train_isenzyme(enzyme_X, enzyme_Y.enzyme_label, model_file=file_isenzyme_model)

XGBoost模型训练完成


In [ ]:
eclablefile='../../data/dict_ec_label.npy'
featurefile='../../data/sprot_unirep.feather'

dict_ec_label, dict_label_ec = train.load_ec_lable_dict(eclablefile)
train_data = train.load_train_file(split_date= DATE_TRAIN, cnx_ecnumber=cnx_ecnumber, type='ec')

#添加训练标签
train_data_labeled  = train.add_ec_label(train_data, dict_ec_label)

unirep_data = train.load_unirep(featurefile)

res_data = train_data_labeled.merge(unirep_data, on='id', how='left')